In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import pandas as pd
import numpy as np
import json
import torch
import re
import ast

In [ ]:
model_name = "Qwen/Qwen2.5-14B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="mps"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)



In [ ]:
system_prompt = "You are an expert at multiple choice simple general knowledge questions by step-by-step reason and return output in json format"

user_prompt_template = """{question}
{options}
{labels}

Solve the above multiple choice simple general knowledge question by breaking it down step by step.

Follow this format strictly:
"Reason:" [...]
"Answer:" [Select only one from the labels] 
"""


In [ ]:
path = '../openbookqa/similarity_generate_questions/'
files = os.listdir(path)
if 'DS_Store' in files:
    files.remove('DS_Store')
len(files)

In [ ]:
with open(path + files[10], 'r') as f:
    data = json.load(f)

In [ ]:
data

In [ ]:
data['choices']['text'][0]

In [ ]:
data['generated_question'][1][13:-1]

In [ ]:
data['generated_question'][0][14:-2]

In [ ]:
data.keys()

In [ ]:
    
def save_pred(question, choices, answer, reason, GT_answer, generated_question, evaluation, question_num, ques_similarity_value, response, path):
    #response = response.replace('"', "'")
    #response = response.replace('\n', "'")
    
    try: 
        print('try')
        reason_start = re.search(r'"Reason":', response, re.DOTALL).span()[0]
        reason_end = re.search(r'"Reason":', response, re.DOTALL).span()[1]
        answer_start = re.search(r'"Answer":', response, re.DOTALL).span()[0]
        answer_end = re.search(r'"Answer":', response, re.DOTALL).span()[1]
    except:
        reason_start = re.search(r"Reason:", response, re.DOTALL).span()[0]
        reason_end = re.search(r"Reason:", response, re.DOTALL).span()[1]
        answer_start = re.search(r"Answer:", response, re.DOTALL).span()[0]
        answer_end = re.search(r"Answer:", response, re.DOTALL).span()[1]

    reason_text = response[reason_start:answer_start]
    reason_lines = [line.strip() for line in reason_text.strip().split("\n") if line.strip()]
    
    answer_text = response[answer_start:]
    answer_str = answer_text.replace("Answer:", "").strip()
    

    result = {
            "question":question,
            "choices":choices,
            "answer":answer,
            "reason":reason,
            "GT_answer":GT_answer,
            "generated_question":generated_question,
            "evaluation":evaluation,
            "question_num":question_num,
            "ques_similarity_value":ques_similarity_value,
            "pred_reason": reason_lines,
            "pred_answer": answer_str,
            
        }
    with open(path, "w") as f:
        json.dump(result, f, indent=2)


In [ ]:

for num, file_name in enumerate(files):
    if num < 454:
        continue
    print(num)
    with open(path + file_name, 'r') as f:
        data = json.load(f)
        
    reason = data['reason']
    question = data['question']
    choices = data['choices']
    answer = data['answer']
    GT_answer = data['GT_answer']
    generated_question = data['generated_question']
    evaluation = data['evaluation']
    question_num = data['question_num']
    ques_similarity_value = data['ques_similarity_value']
    
    max_sim = max(ques_similarity_value)
    index = ques_similarity_value.index(max_sim)
    if index == 0:
        q = data['generated_question'][0][14:-2]
        c = data['generated_question'][1][13:-1]
    elif index == 1:
        q = data['generated_question'][2][14:-2]
        c = data['generated_question'][3][13:-1]
    elif index == 2:
        q = data['generated_question'][4][14:-2]
        c = data['generated_question'][5][13:-1]
    
    c = c.split(',')
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_template.format(question=q, options = c, labels = data['choices']['label'])}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1024
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    p = '../openbookqa/answer_reason_generated_question/' + file_name
    save_pred(question, choices, answer, reason, GT_answer, generated_question, evaluation, question_num, ques_similarity_value, response, p)
    
    

In [ ]:
response = response.replace('"', "'")
response = response.replace('\n', "'")

In [ ]:
response

In [ ]:
reason_start = re.search(r'"Reason":', response, re.DOTALL).span()[1]

In [ ]:
reason_start

In [ ]:
reason_start = re.search(r'"Reason":', response, re.DOTALL).span()[0]